
# Exploratory Data Analysis Using Azure Blob Storage

This notebook performs exploratory data analysis (EDA) on CSV files stored in the Azure Blob Storage account `globalmartmlsa` under the `source` container. The analysis covers sales, inventory, customer behavior, and competitor data, and demonstrates how to connect to Azure Blob Storage using access keys.


In [ ]:

# 📦 Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from azure.storage.blob import BlobServiceClient
import io

# Set visualization style
sns.set(style='whitegrid')

# 🔐 Azure Blob Storage Credentials
account_name = "globalmartmlsa"
account_key = "YOUR_ACCESS_KEY_HERE"  # 🔒 Replace with your actual access key
container_name = "source"

# Set up BlobServiceClient
blob_service_client = BlobServiceClient(
    f"https://{account_name}.blob.core.windows.net",
    credential=account_key
)
container_client = blob_service_client.get_container_client(container_name)

# 📂 Function to Load CSV from Blob Storage
def load_csv_from_blob(blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    stream = blob_client.download_blob().readall()
    return pd.read_csv(io.BytesIO(stream))

# 📁 Load Datasets
sales_data = load_csv_from_blob("Sales Data.csv")
inventory_data = load_csv_from_blob("Inventory Data.csv")
customer_behavior_data = load_csv_from_blob("Daily Customer Behavior.csv")
competitor_data = load_csv_from_blob("Competitor Pricing Data.csv")

# 🕓 Parse and Clean Dates
sales_data['Date'] = pd.to_datetime(sales_data['TransactionDate'], errors='coerce')
inventory_data['Date'] = pd.to_datetime(inventory_data['Date'], errors='coerce')
customer_behavior_data['Date'] = pd.to_datetime(customer_behavior_data['Date'], errors='coerce')
competitor_data['Date'] = pd.to_datetime(competitor_data['Date'], errors='coerce')

# Check for parsing errors
print("Sales Data - Null Dates:", sales_data['Date'].isnull().sum())
print("Inventory Data - Null Dates:", inventory_data['Date'].isnull().sum())
print("Customer Behavior Data - Null Dates:", customer_behavior_data['Date'].isnull().sum())
print("Competitor Data - Null Dates:", competitor_data['Date'].isnull().sum())

# 🧹 Standardize String Fields
for df in [inventory_data, sales_data, competitor_data]:
    if 'FC_ID' in df.columns:
        df['FC_ID'] = df['FC_ID'].astype(str).str.strip().str.upper()
    if 'Brand' in df.columns:
        df['Brand'] = df['Brand'].astype(str).str.strip().str.upper()

# 🔄 Merge Datasets
print("✅ Inventory rows:", len(inventory_data))

if 'FC_ID' in inventory_data.columns and 'FC_ID' in sales_data.columns:
    merged_df = pd.merge(inventory_data, sales_data, on=['Date', 'FC_ID'], how='left')
else:
    merged_df = pd.merge(inventory_data, sales_data, on='Date', how='left')
print("✅ After sales merge:", merged_df.shape)

merged_df = pd.merge(merged_df, customer_behavior_data, on='Date', how='left')
print("✅ After customer behavior merge:", merged_df.shape)

if 'Brand' in competitor_data.columns and 'Brand' in merged_df.columns:
    merged_df = pd.merge(merged_df, competitor_data, on=['Date', 'Brand'], how='left')
else:
    merged_df = pd.merge(merged_df, competitor_data, on='Date', how='left')
print("✅ Final merged rows:", merged_df.shape)

# 👁️ Preview Final Output
print(merged_df.head())
